In [1]:
abstract type ComputableSet end

In [2]:
Base.in(x, X::ComputableSet) = error("should define Base.in for any subtype of ComputableSet")

In [33]:
abstract type 𝔽 end

struct Vec𝔽 <: 𝔽
  elems::Vector{Any}
end

Base.:(∈)(a, A::Vec𝔽) = a ∈ A.elems

Base.iterate(A::Vec𝔽) = iterate(A.elems)
Base.iterate(A::Vec𝔽, k) = iterate(A.elems, k)
Base.length(A::Vec𝔽) = length(A.elems)

struct Int𝔽 <: 𝔽
  n::Int
end

Base.:(∈)(a, A::Int𝔽) = 1 <= a <= A.n

Base.iterate(A::Int𝔽) = iterate(1:A.n)
Base.iterate(A::Int𝔽, k) = iterate(1:A.n, k)

struct 𝔽Mor
  dom::𝔽
  codom::𝔽
  vals::Dict{Any,Any}
end

(f::𝔽Mor)(x) = f.vals[x]

struct Int𝔽Mor
  dom::Int𝔽
  codom::Int𝔽
  vals::Vector{Int}
end

(f::Int𝔽Mor)(i) = f.vals[i]

isvalid(f::𝔽Mor) =
  all(x ∈ keys(f.vals) && f(x) ∈ f.codom for x in f.dom)

isvalid (generic function with 1 method)

In [4]:
struct FiniteSet <: ComputableSet
    X::𝔽
end

In [5]:
Base.in(x, χ::FiniteSet) = x ∈ χ.X

In [6]:
struct TypeSet <: ComputableSet
    T::Type
end

In [7]:
Base.in(x, χ::TypeSet) = x isa χ.T

In [8]:
3 ∈ TypeSet(Int)

true

In [9]:
:a ∈ TypeSet(Float64)

false

In [10]:
abstract type EnumerableSet <: ComputableSet end

In [11]:
struct UnionSet <: ComputableSet
    A::ComputableSet
    B::ComputableSet
end

In [12]:
Base.in(x, χ::UnionSet) = (x in χ.A) || (x in χ.B)

In [13]:
struct IntersectionSet <: ComputableSet
    A::ComputableSet
    B::ComputableSet
end

Base.in(x, χ::IntersectionSet) = (x in χ.A) && (x in χ.B)

In [14]:
3 ∈ IntersectionSet(TypeSet(Int), FiniteSet(Vec𝔽([:a,3,9])))

true

In [15]:
struct ProductSet <: ComputableSet
    A::ComputableSet
    B::ComputableSet
end

Base.in(x, χ::ProductSet) = (x isa Tuple) && (length(x) == 2) && (x[1] in χ.A) && (x[2] in χ.B)

In [16]:
(2,3) ∈ ProductSet(FiniteSet(Vec𝔽([2,3])),FiniteSet(Vec𝔽([2,3])))

true

In [17]:
struct Left
    val::Any
end

struct Right
    val::Any
end

struct SumSet <: ComputableSet
    A::ComputableSet
    B::ComputableSet
end

Base.in(x::Left, χ::SumSet) = x.val ∈ χ.A
Base.in(x::Right, χ::SumSet) = x.val ∈ χ.B

In [18]:
Right("foo") ∈ SumSet(TypeSet(Int), TypeSet(String))

true

In [19]:
struct SingletonSet <: ComputableSet
    val::Any
end

Base.in(x, χ::SingletonSet) = x == χ.val

In [30]:
struct PredicatedSet <: ComputableSet
    T::Type
    p::Any
end

Base.in(x, χ::PredicatedSet) = (x isa χ.T) && χ.p(x)    

BoundsError: BoundsError: attempt to access 69-codeunit String at index [1:70]

BoundsError: BoundsError: attempt to access 69-codeunit String at index [1:70]

In [23]:
abstract type SmallCategory end

In [25]:
Ob(c::SmallCategory) = error()
Hom(c::SmallCategory, x, y) = begin
    @assert x ∈ Ob(c)
    @assert y ∈ Ob(c)
    error()
end

Hom (generic function with 2 methods)

In [26]:
compose(c::SmallCategory, x, y, z, f, g) = begin
    @assert x ∈ Ob(c)
    @assert y ∈ Ob(c)
    @assert z ∈ Ob(c)
    @assert f ∈ Hom(c, x, y)
    @assert g ∈ Hom(c, y, z)
    error()
end

compose (generic function with 1 method)

In [27]:
struct FinSet <: Category
end

In [39]:
Ob(::FinSet) = TypeSet(𝔽)
Hom(::FinSet, x, y) = PredicatedSet(𝔽Mor, f -> (isvalid(f) && f.dom == x && f.codom == y))

Hom (generic function with 3 methods)

In [35]:
A = Vec𝔽([1,2,3]); B = Vec𝔽([3,4])
f = 𝔽Mor(A,B,Dict(1=>3,2=>3,3=>4))

𝔽Mor(Vec𝔽(Any[1, 2, 3]), Vec𝔽(Any[3, 4]), Dict{Any, Any}(2 => 3, 3 => 4, 1 => 3))

In [38]:
f ∈ Hom(FinSet(), A, B)

true

In [41]:
compose(c::FinSet, x, y, z, f, g) = begin
    @assert x ∈ Ob(c)
    @assert y ∈ Ob(c)
    @assert z ∈ Ob(c)
    @assert f ∈ Hom(c, x, y)
    @assert g ∈ Hom(c, y, z)
    𝔽Mor(f.dom, g.codom, Dict(a => g(f(a)) for a in f.dom))
end

compose (generic function with 2 methods)

In [42]:
struct MatrixCat <: Category
end

In [43]:
Ob(::MatrixCat) = PredicatedSet(Int, x -> x >= 0)

Ob (generic function with 4 methods)

In [58]:
Hom(c::MatrixCat, x, y) = begin
    @assert x ∈ Ob(c)
    @assert y ∈ Ob(c)
    PredicatedSet(AbstractMatrix, m -> size(m)[1] == x && size(m)[2] == y)
end

Hom (generic function with 4 methods)

In [47]:
[1 2; 2 3; 3 4] ∈ Hom(MatrixCat(), 3, 2)

true

In [48]:
compose(c::MatrixCat, x, y, z, f, g) = begin
    @assert x ∈ Ob(c)
    @assert y ∈ Ob(c)
    @assert z ∈ Ob(c)
    @assert f ∈ Hom(c, x, y)
    @assert g ∈ Hom(c, y, z)
    f * g
end

compose (generic function with 3 methods)

In [51]:
compose(MatrixCat(), 3, 2, 3, [1 2; 2 3; 3 4], collect([1 2; 2 3; 3 4]'))

3×3 Matrix{Int64}:
  5   8  11
  8  13  18
 11  18  25

In [52]:
[1 2]'

2×1 adjoint(::Matrix{Int64}) with eltype Int64:
 1
 2

In [55]:
dot(v,w) = v * w'

dot (generic function with 1 method)

In [56]:
dot([2 3], [4 5])

1×1 Matrix{Int64}:
 23

In [61]:
using LinearAlgebra

identity(c::MatrixCat, x) = begin
    @assert x ∈ Ob(c)
    I(x)
end

identity (generic function with 1 method)

In [63]:
identity(MatrixCat(), 4) ∈ Hom(MatrixCat(), 4, 4)

true